In [1]:
import pandas as pd
from openai import OpenAI
import json
import re
from pydantic import BaseModel
import time
from tqdm import tqdm
import os
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from pydantic import BaseModel
import os
import openai
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score
from datasets import load_dataset
import pandas as pd
from openai import OpenAI
import json
import re
from pydantic import BaseModel
import time
from tqdm import tqdm
import os
import requests
import json
import os
import getpass
from tqdm import tqdm
import pandas as pd
import time
import requests, json, os, time
from tqdm.notebook import tqdm
import pandas as pd
# Cell for implementing the recursive reasoning summarizer loop
import pandas as pd
import openai
import json
import re
import time
import requests
import random
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('data/aime/aime_2024_completions_updated.csv')
df

,ID,Problem,Solution,Answer,model_completion,extracted_answer
0,2024-II-4,"Let $x,y$ and $z$ be positive real numbers tha...","Denote $\log_2(x) = a$, $\log_2(y) = b$, and $...",33,"<think>\nOkay, so I have this problem where I ...",33
1,2024-II-12,"Let $O(0,0), A(\tfrac{1}{2}, 0),$ and $B(0, \t...",Begin by finding the equation of the line $\ov...,23,"<think>\nOkay, so I need to solve this geometr...",23
2,2024-I-4,Jen enters a lottery by picking $4$ distinct n...,This is a conditional probability problem. Bay...,116,"<think>\nOkay, let's try to figure out this pr...",116
3,2024-I-3,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,"<think>\nOkay, so Alice and Bob are playing th...",809
4,2024-I-8,Eight circles of radius $34$ are sequentially ...,Draw an altitude from both end circles of the ...,197,"<think>\nAlright, so I need to solve this geom...",2926
5,2024-I-12,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...,We will denote $h(x)=4g(f(x))$ for simplicity....,385,"<think>\nOkay, I need to solve this problem wh...",16
6,2024-I-11,Each vertex of a regular octagon is independen...,Let $r$ be the number of red vertices and $b$ ...,371,"<think>\nAlright, let me try to figure out thi...",279
7,2024-II-11,Find the number of triples of nonnegative inte...,$ab(a+b)+bc(b+c)+ac(a+c)=300(ab+bc+ac)-3abc=60...,601,"<think>\nOkay, so I need to find the number of...",601
8,2024-I-2,"There exist real numbers $x$ and $y$, both gre...","By properties of logarithms, we can simplify t...",25,"<think>\nOkay, let me try to figure out this p...",25
9,2024-II-6,Alice chooses a set $A$ of positive integers. ...,Let $k$ be one of the elements in Alice's set ...,55,"<think>\nOkay, let's try to figure out this pr...",55


In [3]:

# Function to summarize reasoning trace using GPT-4o
def summarize_reasoning(reasoning_trace):
    """
    Summarize the model's reasoning trace into a more concise form while preserving critical reasoning steps.
    """
    openai_client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    summarizer_prompt = [
        {"role": "system", "content": """You are an expert at summarizing mathematical reasoning traces.
        Your goal is to produce concise but complete summaries of reasoning traces that:
        1. Preserve all key reasoning steps and intermediate calculations
        2. Highlight potential errors or invalid assumptions in the original reasoning
        3. Identify what paths of reasoning were explored and abandoned
        4. Maintain the logical flow and dependencies between steps
        5. Avoid introducing new reasoning or calculations not present in the original trace
        6. Format the summary in a clear, step-by-step manner
        7. Include key equations and numerical results with proper notation
        
        Your summary should be detailed enough that another model could understand the full reasoning process,
        but concise enough to eliminate redundancy and verbose explanations."""},
        {"role": "user", "content": f"""Below is a reasoning trace from a math problem. 
        Summarize this reasoning trace while preserving all key steps, intermediate calculations, 
        and potential errors in the original reasoning. Return just the summary.
        
        REASONING TRACE:
        {reasoning_trace}
        
        SUMMARY:"""}
    ]
    
    for attempt in range(3):
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4o",
                messages=summarizer_prompt,
                temperature=0
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Summarization attempt {attempt+1} failed: {e}")
            if attempt < 2:
                print(f"Retrying in {(2 ** attempt) * 2} seconds...")
                time.sleep((2 ** attempt) * 2)
            else:
                print("Max retries reached. Returning original reasoning.")
                return reasoning_trace

# Function to get continued reasoning from DeepSeek-R1
def continue_reasoning(question, reasoning_so_far, fireworks_key, model="accounts/fireworks/models/deepseek-r1", max_tokens=20000):
    """
    Continue reasoning by submitting a new completion request with the summarized reasoning
    """
    # Generate a random continuation phrase
    continuation_phrases = [
        "Let me continue thinking more deeply about this problem.",
        "I need to explore this problem further to find the correct solution.",
        "Let me reconsider my approach and continue reasoning.",
        "I should take a different angle and continue my reasoning.",
        "I'll extend my reasoning to ensure I arrive at the correct answer.",
        "I need to think about this more carefully."
    ]
    continuation_phrase = random.choice(continuation_phrases)
    
    # Construct prompt with original question, opening <think> tag, summarized reasoning, and continuation
    prompt = f"Solve the following question: \n\n'{question}'\n\n<think>\n{reasoning_so_far}\n\n{continuation_phrase}"
    
    # API constants
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {fireworks_key}"
    }
    url = "https://api.fireworks.ai/inference/v1/completions"
    
    # Make API call with retry logic
    for attempt in range(3):
        try:
            response = requests.post(
                url, 
                headers=headers,
                json={
                    "model": model,
                    "max_tokens": max_tokens,
                    "temperature": 0.7,
                    "prompt": prompt
                },
            )
            
            if response.status_code == 200:
                new_completion = response.json().get("choices", [{}])[0].get("text", "")
                # Combine existing reasoning, continuation phrase, and new completion
                combined_reasoning = f"<think>\n{reasoning_so_far}\n\n{continuation_phrase}{new_completion}"
                return combined_reasoning
            elif response.status_code == 429:  # Rate limit
                wait_time = (2 ** attempt) * 2
                print(f"Rate limited. Waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"API error: {response.status_code}, {response.text}")
                if attempt == 2:
                    return reasoning_so_far  # Return original reasoning if all attempts fail
        except Exception as e:
            print(f"Error: {str(e)}")
            time.sleep(2)
            if attempt == 2:
                return reasoning_so_far  # Return original reasoning if all attempts fail

# Function to extract reasoning from between <think> and </think> tags
def extract_thinking(completion):
    """Extract the reasoning part between <think> and </think> tags"""
    if pd.isna(completion):
        return ""
    
    # Find content between <think> and </think>
    pattern = r'<think>(.*?)</think>'
    match = re.search(pattern, completion, re.DOTALL)
    
    if match:
        return match.group(1).strip()
    else:
        # If no closing tag, take everything after <think>
        start_idx = completion.find("<think>")
        if start_idx != -1:
            return completion[start_idx + 7:].strip()
        else:
            return completion.strip()

# Extract answer from boxed notation
def extract_boxed_answer(text):
    """Extract answer from \boxed{X} notation"""
    if pd.isna(text):
        return ""
    
    pattern = r'\\boxed\{([^}]+)\}'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1).strip()
    else:
        return ""

# Main function to run the recursive reasoning loop
def run_recursive_reasoning_loop(df, fireworks_key, answer_column='Answer', max_iterations=2):
    """
    Process each row, check if extracted_answer differs from answer, and run reasoning summarizer loop
    
    Args:
        df: DataFrame with model_completion and extracted_answer columns
        fireworks_key: API key for Fireworks
        answer_column: Column name containing the correct answer
        max_iterations: Maximum iterations of summarize-continue loop
        
    Returns:
        Updated DataFrame with new_completion and new_extracted_answer columns
    """
    # Make sure required columns exist
    if 'extracted_answer' not in df.columns:
        print("Adding extracted_answer column...")
        df['extracted_answer'] = df['model_completion'].apply(extract_boxed_answer)
    
    # Add new columns
    df['summarized_completion'] = None
    df['summarized_extracted_answer'] = None
    df['iterations'] = 0
    
    # Find rows with mismatched answers
    mismatch_indices = []
    for idx in range(len(df)):
        extracted = str(df.loc[idx, 'extracted_answer']).strip() if not pd.isna(df.loc[idx, 'extracted_answer']) else ""
        answer = str(df.loc[idx, answer_column]).strip() if not pd.isna(df.loc[idx, answer_column]) else ""
        
        if extracted != answer and not pd.isna(df.loc[idx, 'model_completion']):
            mismatch_indices.append(idx)
    
    print(f"Found {len(mismatch_indices)} rows with mismatched answers")
    
    # Process each mismatched row
    for idx in tqdm(mismatch_indices, desc="Processing mismatched answers"):
        question = df.loc[idx, 'Problem']
        original_completion = df.loc[idx, 'model_completion']
        
        # Extract the reasoning from the original completion
        reasoning = extract_thinking(original_completion)
        
        if not reasoning:
            print(f"No reasoning found for row {idx}, skipping")
            continue
        
        # Current reasoning starts with original reasoning
        current_reasoning = reasoning
        current_completion = original_completion
        
        # Track if we found a boxed answer in any iteration
        found_boxed_answer = False
        
        # Run up to max_iterations of summarize-continue loop
        for iteration in range(max_iterations):
            print(f"\nRow {idx}, Iteration {iteration+1}:")
            
            # Step 1: Summarize the reasoning
            print("Summarizing reasoning...")
            summarized_reasoning = summarize_reasoning(current_reasoning)
            print(f"Summarized reasoning: {summarized_reasoning}")

            
            # Step 2: Continue reasoning with DeepSeek-R1
            print("Continuing reasoning with DeepSeek-R1...")
            new_completion = continue_reasoning(question, summarized_reasoning, fireworks_key)
            
            # Extract the new reasoning (everything after our continuation prompt)
            current_reasoning = new_completion
            current_completion = new_completion
            
            # Check if we now have a boxed answer
            new_extracted = extract_boxed_answer(current_completion)
            
            if new_extracted:
                found_boxed_answer = True
                print(f"Found answer: {new_extracted}")
                break
        
        print(f"\nFinal completion for row {idx}: {current_completion}")
        # Save results
        df.loc[idx, 'new_completion'] = current_completion
        df.loc[idx, 'new_extracted_answer'] = extract_boxed_answer(current_completion)
        df.loc[idx, 'iterations'] = iteration + 1
    
    # Calculate statistics
    fixed_count = 0
    for idx in mismatch_indices:
        new_extracted = str(df.loc[idx, 'new_extracted_answer']).strip() if not pd.isna(df.loc[idx, 'new_extracted_answer']) else ""
        answer = str(df.loc[idx, answer_column]).strip() if not pd.isna(df.loc[idx, answer_column]) else ""
        
        if new_extracted == answer:
            fixed_count += 1
    
    print(f"\nResolved {fixed_count} out of {len(mismatch_indices)} mismatched answers ({fixed_count/len(mismatch_indices)*100:.2f}%)")
    
    return df

# Run the recursive reasoning loop
if 'fireworks_key' not in globals():
    fireworks_key = os.environ.get("FIREWORKS_API_KEY")
    if not fireworks_key:
        fireworks_key = input("Enter your Fireworks API key: ")

# Uncomment to run the full process
df_updated = run_recursive_reasoning_loop(df, fireworks_key, answer_column='Answer', max_iterations=2)

# To save results
df_updated.to_csv('data/aime/recursive_reasoning_results.csv', index=False)

Found 7 rows with mismatched answers


Processing mismatched answers:   0%|          | 0/7 [00:00<?, ?it/s]


Row 4, Iteration 1:
Summarizing reasoning...
Summarized reasoning: 1. **Problem Understanding**: The problem involves a triangle ABC with an inradius expressed as \( \frac{m}{n} \), where \( m \) and \( n \) are coprime. Two configurations of circles are given: eight circles of radius 34 and 2024 circles of radius 1, both arranged in the same manner within the triangle.

2. **Initial Assumptions**: The reasoning explores the idea that the circles form a chain from side AB to side BC, with the first and last circles tangent to these sides, respectively. The inradius is initially thought to relate to the sum of the radii of these circles.

3. **Scaling and Ratios**: The reasoning considers the scaling of the triangle or the configurations, noting that the number of circles increases by a factor of 253 (from 8 to 2024), while the radius decreases by a factor of 34 (from 34 to 1). This suggests a potential inverse relationship between the number of circles and their radii.

4. **Geometric

In [4]:
df

,ID,Problem,Solution,Answer,model_completion,extracted_answer,summarized_completion,summarized_extracted_answer,iterations,new_completion,new_extracted_answer
0,2024-II-4,"Let $x,y$ and $z$ be positive real numbers tha...","Denote $\log_2(x) = a$, $\log_2(y) = b$, and $...",33,"<think>\nOkay, so I have this problem where I ...",33,None,None,0,NaN,NaN
1,2024-II-12,"Let $O(0,0), A(\tfrac{1}{2}, 0),$ and $B(0, \t...",Begin by finding the equation of the line $\ov...,23,"<think>\nOkay, so I need to solve this geometr...",23,None,None,0,NaN,NaN
2,2024-I-4,Jen enters a lottery by picking $4$ distinct n...,This is a conditional probability problem. Bay...,116,"<think>\nOkay, let's try to figure out this pr...",116,None,None,0,NaN,NaN
3,2024-I-3,Alice and Bob play the following game. A stack...,Let's first try some experimentation. Alice ob...,809,"<think>\nOkay, so Alice and Bob are playing th...",809,None,None,0,NaN,NaN
4,2024-I-8,Eight circles of radius $34$ are sequentially ...,Draw an altitude from both end circles of the ...,197,"<think>\nAlright, so I need to solve this geom...",2926,None,None,1,<think>\n1. **Problem Understanding**: The pro...,2927
5,2024-I-12,Define $f(x)=|| x|-\tfrac{1}{2}|$ and $g(x)=||...,We will denote $h(x)=4g(f(x))$ for simplicity....,385,"<think>\nOkay, I need to solve this problem wh...",16,None,None,1,<think>\n1. **Problem Setup**: The task is to ...,16
6,2024-I-11,Each vertex of a regular octagon is independen...,Let $r$ be the number of red vertices and $b$ ...,371,"<think>\nAlright, let me try to figure out thi...",279,None,None,1,<think>\n1. **Problem Understanding**: The tas...,279
7,2024-II-11,Find the number of triples of nonnegative inte...,$ab(a+b)+bc(b+c)+ac(a+c)=300(ab+bc+ac)-3abc=60...,601,"<think>\nOkay, so I need to find the number of...",601,None,None,0,NaN,NaN
8,2024-I-2,"There exist real numbers $x$ and $y$, both gre...","By properties of logarithms, we can simplify t...",25,"<think>\nOkay, let me try to figure out this p...",25,None,None,0,NaN,NaN
9,2024-II-6,Alice chooses a set $A$ of positive integers. ...,Let $k$ be one of the elements in Alice's set ...,55,"<think>\nOkay, let's try to figure out this pr...",55,None,None,0,NaN,NaN
